# [실습 프로젝트]

### 실습 목표

1. 아래 샘플 문서들을 벡터 저장소에 저장하는 코드를 작성합니다.
   - 적절한 벡터 저장소 선택
   - 임베딩 모델 설정
   - 문서 구조 설계 (metadata 정의)

2. 벡터 저장소를 사용하여 다음 기능을 구현합니다.
   - 새로운 문서 추가
   - 문서 삭제
   - 문서 검색: 유사도 점수 계산, 메타데이터 기반 필터링 등

### 실습 단계

- **1단계: 벡터 저장소 초기화**
   - Chroma, FAISS, Pinecone 중 하나를 선택하고 이유를 설명하세요

- **2단계: 문서 저장**
   - 제공된 샘플 문서를 Document 객체로 변환하세요
   - metadata 구조를 설계하세요 (type, author 포함)

- **3단계: 문서 관리**
   - 새로운 문서 1개를 추가하세요
   - 특정 문서 1개를 삭제하세요

- **4단계: 문서 검색 구현**
   - [ ] 기본 유사도 검색
   - [ ] 메타데이터 필터링
   - [ ] 점수 포함 검색

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
import os
from glob import glob
from pprint import pprint
import json


/Users/hosunglee/test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
documents = [
    {"content": "태양광 발전시설 이격거리: 주거지역으로부터 100m 이내 설치 제한 (2024 산업부 지침).", "author": "산업통상자원부", "type": "article", "region": "전국"},
    {"content": "농지 태양광 설치 허가: 영농형 태양광의 경우 최대 20년까지 타용도 일시사용허가 가능.", "author": "농림축산식품부", "type": "article", "region": "전국"},
    {"content": "REC 가중치 규정: 임야 설치 시 가중치 0.5, 지붕 설치 시 1.5 부여.", "author": "에너지관리공단", "type": "article", "region": "전국"},
    {"content": "안전관리자 선임: 설비 용량 1,000kW 이상인 경우 전기안전관리자 상주 필수.", "author": "전기안전공사", "type": "article", "region": "전국"},
    {"content": "계통연계 비용: 100kW 미만 소규모 발전소의 경우 한전 부담으로 변압기 보강 실시.", "author": "한국전력", "type": "article", "region": "전국"},
    {"content": "변압기 점검: 산가 수치가 0.2mgKOH/g를 초과하면 절연유 교체 및 정밀 진단 권고.", "author": "현장기술팀", "type": "guideline", "region": "전국"},
    {"content": "인버터 E-21 에러: 과전압 보호 작동. DC 전압 입력값 확인 후 수동 리셋 필요.", "author": "A/S센터", "type": "guideline", "region": "전국"},
    {"content": "차단기(ACB) 수명: 정격 차단 횟수 1만 회 도달 시 메커니즘 부품 교체 필수.", "author": "제조사지침", "type": "guideline", "region": "전국"},
    {"content": "접지 저항 기준: 태양광 발전 시스템 제3종 접지 공사 시 저항값 100Ω 이하 유지.", "author": "기술표준", "type": "guideline", "region": "전국"},
    {"content": "ESS 화재 예방: 리튬이온 배터리 SOC를 상시 80% 이하로 운용 권장.", "author": "소방방재청", "type": "guideline", "region": "전국"},
    {"content": "패널 세척 효과: 건기 시 표면 오염을 세척하면 발전 효율이 약 5~12% 상승함.", "author": "운영데이터", "type": "guideline", "region": "전국"},
    {"content": "그늘(Shading) 영향: 모듈 일부에 그늘 발생 시 핫스팟 현상 주의.", "author": "안전점검원", "type": "guideline", "region": "전국"},
    {"content": "바이패스 다이오드: 셀 결함 발생 시 전류를 우회시켜 전체 모듈 출력 저하 방지.", "author": "설계매뉴얼", "type": "guideline", "region": "전국"},
    {"content": "양면형(Bifacial) 모듈: 지면 반사광을 이용해 후면에서 추가 15% 발전 가능.", "author": "R&D센터", "type": "guideline", "region": "전국"},
    {"content": "기온과 효율: 셀 온도가 25도에서 1도 상승할 때마다 효율은 약 0.5%씩 감소함.", "author": "기상연구소", "type": "guideline", "region": "전국"},
    {"content": "VPP(가상발전소): 소규모 분산 자원을 ICT 기술로 통합해 하나의 발전소처럼 운영.", "author": "미래전략실", "type": "article", "region": "전국"},
    {"content": "스마트 그리드: 전력망에 IT를 접목해 에너지 효율을 최적화하는 지능형 전력망.", "author": "미래전략실", "type": "article", "region": "전국"},
    {"content": "안티 아일랜딩: 계통 정전 시 인버터가 즉시 가동을 멈춰 작업자 안전을 확보하는 기능.", "author": "표준규격", "type": "article", "region": "전국"},
    {"content": "MPPT 제어: 일사량에 따라 태양광 모듈의 최대 전력점을 추적하여 효율 극대화.", "author": "제어공학", "type": "guideline", "region": "전국"},
    {"content": "염해 방지 설계: 해안가 1km 이내 설치 시 내염해성 부품 사용 권장.", "author": "시공가이드", "type": "guideline", "region": "전국"},
    {"content": "산지 태양광 경사도 규정: 평균 경사도 15도 이하인 경우에만 설치 허가가 가능하다.", "author": "산림청", "type": "article", "region": "전국"},
    {"content": "신재생에너지 의무공급량(RPS): 발전사업자는 총 발전량의 일정 비율 이상을 신재생에너지로 공급해야 함.", "author": "산업통상자원부", "type": "article", "region": "전국"},
    {"content": "분산에너지 활성화 특별법: 지역 내 생산된 에너지를 지역 내에서 소비하도록 유도하는 체계.", "author": "국회", "type": "article", "region": "전국"},
    {"content": "소형태양광 고정가격계약(한국형 FIT): 30kW 미만 소규모 사업자의 안정적 수익 보장 제도.", "author": "에너지공단", "type": "article", "region": "전국"},
    {"content": "전력거래소(KPX) 급전 지시: 전력 수급 불균형 시 신재생에너지 출력 제한을 지시할 수 있음.", "author": "전력거래소", "type": "article", "region": "전국"},
    {"content": "전기사업법 제67조: 전기설비의 안전을 위해 정기점검을 거부할 경우 과태료가 부과될 수 있음.", "author": "법제처", "type": "article", "region": "전국"},
    {"content": "RE100 캠페인: 기업이 사용하는 전력의 100%를 재생에너지로 충당하겠다는 글로벌 선언.", "author": "RE100위원회", "type": "article", "region": "전국"},
    {"content": "인버터 과열 방지: 통풍구의 먼지를 제거하고 직사광선을 피하기 위한 차양막 설치 권장.", "author": "기술지원팀", "type": "guideline", "region": "전국"},
    {"content": "피뢰 설비 점검: 하절기 낙뢰 피해 방지를 위해 서지 보호 장치(SPD)의 소자 상태 상시 확인.", "author": "안전점검원", "type": "guideline", "region": "전국"},
    {"content": "변압기 소음 대처: 비정상적인 웅웅거리는 소음 발생 시 코어 결합 상태 확인 및 전문가 진단 의뢰.", "author": "유지보수팀", "type": "guideline", "region": "전국"},
    {"content": "패널 균열(Micro-crack) 관리: 드론 열화상 카메라를 이용해 핫스팟 발생 여부를 정기 측정함.", "author": "운영데이터", "type": "guideline", "region": "전국"},
    {"content": "케이블 노후화 방지: UV 차단 코팅이 된 전력 케이블을 사용하여 외부 노출로 인한 피복 손상 방지.", "author": "시공가이드", "type": "guideline", "region": "전국"},
    {"content": "ESS 배터리 냉각: 실내 온도를 25도 내외로 유지하도록 항온항습기 설정을 최적화해야 함.", "author": "소방방재청", "type": "guideline", "region": "전국"},
    {"content": "모듈 각도 최적화: 한국 지형 기준 남향 30~35도 설치 시 연간 발전량이 가장 극대화됨.", "author": "에너지연구소", "type": "guideline", "region": "전국"},
    {"content": "데이터 로거 통신 장애: 통신 모듈 재부팅 및 RS-485 배선 접촉 상태 우선 점검.", "author": "A/S센터", "type": "guideline", "region": "전국"},
    {"content": "자가용 태양광 잉여전력 상계거래: 사용 후 남은 전력을 한전에 이월하여 전기요금 차감 가능.", "author": "한국전력", "type": "guideline", "region": "전국"},
    {"content": "인버터 단독운전 방지 기능: 계통 정전 시 태양광 전력이 전력망으로 역송전되는 것을 차단함.", "author": "표준규격", "type": "guideline", "region": "전국"},
    {"content": "전압 강하 방지: 인버터와 계량기 사이의 거리가 멀 경우 굵은 배선을 사용하여 전력 손실 최소화.", "author": "설계매뉴얼", "type": "guideline", "region": "전국"},
    {"content": "염해 지역 모듈 선택: 해안가 인접 지역은 부식 방지를 위한 C5-M 등급 프레임 사용 권장.", "author": "기술표준", "type": "guideline", "region": "전국"},
    {"content": "변압기 절연 저항 측정: 정기점검 시 500V 또는 1000V 메거를 사용하여 절연 수치 기록 관리.", "author": "전기안전공사", "type": "guideline", "region": "전국"},
    {"content": "농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.", "author": "경기도청", "type": "조례", "region": "경기도"},
    {"content": "태양광 발전시설은 주거지역 경계로부터 최소 100미터 이상 이격하여 설치해야 합니다.", "author": "충청남도청", "type": "조례", "region": "충청남도"},
    {"content": "자연녹지지역 내 태양광 설치는 지목이 잡종지인 경우에 한하여 허용되며, 산림 훼손을 최소화해야 합니다.", "author": "전라남도청", "type": "조례", "region": "전라남도"},
    {"content": "도로변 태양광 설치 시 국도 및 지방도로부터 50미터 이상 이격거리를 유지해야 하며, 경관 심의를 받아야 합니다.", "author": "경상북도청", "type": "조례", "region": "경상북도"},
    {"content": "태양광 발전용량 100kW 이상은 전기사업 허가가 필요하며, 100kW 미만은 소규모 사업으로 신고만으로 가능합니다.", "author": "산업통상자원부", "type": "가이드라인", "region": "전국"},
    {"content": "배전선로 연계 용량은 해당 변전소 여유 용량에 따라 결정되며, 한국전력공사에 사전 접속 신청이 필요합니다.", "author": "한국전력공사", "type": "규정", "region": "전국"},
    {"content": "태양광 설비 설치 후 사용전 검사를 받아야 하며, 전기안전관리자를 선임 또는 위탁해야 합니다.", "author": "한국전기안전공사", "type": "규정", "region": "전국"},
    {"content": "개발제한구역(그린벨트) 내에서는 태양광 발전시설 설치가 원칙적으로 금지되나, 기존 건축물 지붕에 한해 예외적으로 허용됩니다.", "author": "서울특별시청", "type": "조례", "region": "서울특별시"},
    {"content": "경사도 15도 이상의 산지에는 태양광 설치가 불가하며, 산지전용허가를 받은 경우에만 설치 가능합니다.", "author": "강원도청", "type": "조례", "region": "강원도"},
    {"content": "태양광 발전사업 REC(신재생에너지 공급인증서) 가중치는 설치 유형에 따라 0.7~1.5까지 다르게 적용됩니다.", "author": "산업통상자원부", "type": "가이드라인", "region": "전국"},
    {"content": "답(논) 지목의 토지에 태양광 설치 시 농지법에 따른 농지전용부담금이 부과되며, 면적에 따라 비용이 산정됩니다.", "author": "전라북도청", "type": "조례", "region": "전라북도"},
    {"content": "태양광 발전시설 주변 500미터 이내에 학교, 병원, 군사시설이 있는 경우 설치 제한 심의 대상이 됩니다.", "author": "경상남도청", "type": "조례", "region": "경상남도"},
    {"content": "잡종지 지목의 경우 태양광 설치에 가장 유리하며, 별도의 지목변경 없이 발전사업 신고 후 설치 가능합니다.", "author": "에너지공단", "type": "가이드라인", "region": "전국"},
    {"content": "보전관리지역 내 태양광 발전시설은 부지 면적 1만㎡ 이하로 제한되며, 환경영향평가 협의가 필요합니다.", "author": "충청북도청", "type": "조례", "region": "충청북도"},
    {"content": "태양광 모듈 설치각도는 지역 위도에 따라 최적 각도가 다르며, 남향 30~35도가 일반적으로 최적 발전 효율을 나타냅니다.", "author": "한국에너지기술연구원", "type": "가이드라인", "region": "전국"},
]



In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [ ]:
import uuid
from langchain_core.documents import Document
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)


In [ ]:
#transform


doc_objects = [
    Document(
        page_content=doc["content"],
        metadata={
            "type": doc["type"],
            "region": doc["region"],
            "author": doc["author"],
        },
    )
    for doc in documents
]


# UUID 생성
doc_ids = [str(uuid.uuid4()) for _ in range(len(doc_objects))]

# 벡터 저장소 저장
chroma_db.add_documents(documents=doc_objects, ids=doc_ids)


print(f"\n[샘플 문서 확인]")
print(f"내용: {doc_objects[0].page_content}")
print(f"메타데이터: {doc_objects[0].metadata}")



[샘플 문서 확인]
내용: 태양광 발전시설 이격거리: 주거지역으로부터 100m 이내 설치 제한 (2024 산업부 지침).
메타데이터: {'type': 'article', 'region': '전국', 'author': '산업통상자원부'}


In [ ]:
# 새로운 문서 1개 추가
new_doc = Document(
    page_content="수상태양광(저수지, 댐)은 수면 면적의 30% 이하로 설치 가능하며, 수질환경 영향평가가 필요합니다.",
    metadata={
        "type": "조례",
        "region": "전국",
        "author": "환경부",
    },
)
new_id = str(uuid.uuid4())
chroma_db.add_documents(documents=[new_doc], ids=[new_id])


print(f"추가된 문서: {new_doc.page_content}")

추가된 문서: 수상태양광(저수지, 댐)은 수면 면적의 30% 이하로 설치 가능하며, 수질환경 영향평가가 필요합니다.


In [ ]:
# 첫 번째 문서 삭제
delete_id = doc_ids[0]
chroma_db.delete(ids=[delete_id])

In [ ]:
# (1) 기본 유사도 검색
query = "공터에 태양광 설치할 수 있나요?"
results = chroma_db.similarity_search(query, k=3)

print(f"쿼리: {query}\n")
for i, doc in enumerate(results):
    print(f"[{i+1}] {doc.page_content}")
    print(f" 지역: {doc.metadata['region']} | 유형: {doc.metadata['type']}")
    print()


쿼리: 공터에 태양광 설치할 수 있나요?

[1] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
 지역: 경기도 | 유형: 조례

[2] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
 지역: 경기도 | 유형: 조례

[3] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
 지역: 경기도 | 유형: 조례



In [ ]:
# (2) 메타데이터 필터링 (경기도 조례)
query = "태양광 설치 제한 조건"
results = chroma_db.similarity_search(
    query,
    k=3,
    filter={"region": "경기도"}
)

if results:
    for i, doc in enumerate(results):
        print(f"[{i+1}] {doc.page_content}")
        print(f"    지역: {doc.metadata['region']} | 발행: {doc.metadata['author']}")
        print()
else:
    print("검색 결과 없음")

[1] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
    지역: 경기도 | 발행: 경기도청

[2] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
    지역: 경기도 | 발행: 경기도청

[3] 농지에 태양광 발전시설을 설치하려면 농지전용허가를 받아야 하며, 농업진흥구역 내에서는 설치가 불가능합니다.
    지역: 경기도 | 발행: 경기도청



In [ ]:
# (3) 점수 포함 검색
query = "배전선로 연계 용량 신청 방법"
results = chroma_db.similarity_search_with_relevance_scores(query, k=3)

print(f" 쿼리: {query}\n")
for doc, score in results:
    print(f"관련성 점수: {score:.4f}")
    print(f"내용: {doc.page_content}")
    print(f"지역: {doc.metadata['region']} | 유형: {doc.metadata['type']}")
    print()

🔍 쿼리: 배전선로 연계 용량 신청 방법

관련성 점수: 0.6695
내용: 배전선로 연계 용량은 해당 변전소 여유 용량에 따라 결정되며, 한국전력공사에 사전 접속 신청이 필요합니다.
지역: 전국 | 유형: 규정

관련성 점수: 0.6695
내용: 배전선로 연계 용량은 해당 변전소 여유 용량에 따라 결정되며, 한국전력공사에 사전 접속 신청이 필요합니다.
지역: 전국 | 유형: 규정

관련성 점수: 0.6695
내용: 배전선로 연계 용량은 해당 변전소 여유 용량에 따라 결정되며, 한국전력공사에 사전 접속 신청이 필요합니다.
지역: 전국 | 유형: 규정



# 챗봇형태로 구현 , 추후 open api와 연결시도해보겠습니다

In [ ]:
# From dotenv import load_dotenv
# load_dotenv()




import gradio as gr
from typing import Iterator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage

# 1. 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 태양광 발전 설치 전문 상담사입니다.
아래 제공된 문서를 참고하여 질문에 답변해주세요.
문서에 없는 내용은 모른다고 솔직하게 말씀해주세요.

[참고 문서]
{context}
"""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# 2. LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    top_p=0.9,
    presence_penalty=0.3,
    frequency_penalty=0.3,
)

# 3. Retriever 설정
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 4. RAG 체인 구성
chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["input"])),
        "chat_history": lambda x: x["chat_history"],
        "input": lambda x: x["input"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

def solar_bot(message, history):
    history_messages = []
    for human_msg, ai_msg in history:
        history_messages.append(HumanMessage(content=human_msg))
        history_messages.append(AIMessage(content=ai_msg))

    response = chain.invoke({
        "chat_history": history_messages,
        "input": message
    })
    return response

demo = gr.ChatInterface(
    fn=solar_bot,
    title="태양광 설치 상담 챗봇",
    description="태양광 설치 관련 조례, 규정, 기술 가이드를 검색하여 답변하는 시스템.",
    examples=[
        "농지에 태양광 설치할 수 있나요?",
        "경기도 태양광 설치 조례가 궁금해요",
        "배전선로 연계 신청은 어떻게 하나요?",
        "REC 가중치는 어떻게 되나요?",
    ],
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
